In [38]:
using JuMP, HiGHS, DataFrames # Importar as bibliotecas a se usar

In [39]:
mod = Model(HiGHS.Optimizer) # Criação do Modelo do Problema

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [40]:
capacidade_producao = [200000, 50000, 125000, 33500] # Lista com os valores da capacidade de produção
custo_industrial = [42.5, 45, 47.5, 50] # Lista com os valores do custo industrial

4-element Vector{Float64}:
 42.5
 45.0
 47.5
 50.0

In [41]:
custo_transporte = [
    [0.1, 0.6, 0.25, 0.8, 1.2, 0.55],
    [0.5, 0.8, 0.05, 0.2, 0.5, 0.75],
    [0.4, 0.5, 0.7, 0.45, 1, 0.2],
    [0.7, 0.75, 0.5, 0.85, 0.6, 0.9]
] # Lista de listas, cada uma correspondendo aos preços de transporte de cada fábrica para os armazéns, seguindo indexação
# Lista 1 - De Nocal para cada um dos armazéns, e por aí adiante

4-element Vector{Vector{Float64}}:
 [0.1, 0.6, 0.25, 0.8, 1.2, 0.55]
 [0.5, 0.8, 0.05, 0.2, 0.5, 0.75]
 [0.4, 0.5, 0.7, 0.45, 1.0, 0.2]
 [0.7, 0.75, 0.5, 0.85, 0.6, 0.9]

In [42]:
procura = [245000, 30000, 58387, 58656, 25000, 38700]

6-element Vector{Int64}:
 245000
  30000
  58387
  58656
  25000
  38700

In [43]:
@variable(mod, 0<=x[i=1:4]) # Criação das variáveis relativas à produção da cerveja

4-element Vector{VariableRef}:
 x[1]
 x[2]
 x[3]
 x[4]

In [44]:
@variable(mod, 0<=u[i=1:4, j=1:6]) # Criação das variáveis relativas ao transporte
# Têm nome 'u' pelo nome 'x' já estar presente nas variáveis de produção

4×6 Matrix{VariableRef}:
 u[1,1]  u[1,2]  u[1,3]  u[1,4]  u[1,5]  u[1,6]
 u[2,1]  u[2,2]  u[2,3]  u[2,4]  u[2,5]  u[2,6]
 u[3,1]  u[3,2]  u[3,3]  u[3,4]  u[3,5]  u[3,6]
 u[4,1]  u[4,2]  u[4,3]  u[4,4]  u[4,5]  u[4,6]

In [45]:
@variable(mod, 0 <= e[i=1:4])  # Variável para a produção em regime extraordinária
# Usada para tornar o modelo linear, permitindo assim o seu cálculo, têm o nome "e" de "extraordinário"

4-element Vector{VariableRef}:
 e[1]
 e[2]
 e[3]
 e[4]

In [46]:
@objective(mod, Min,
    sum(custo_industrial[i] * x[i] + 
        1.1 * custo_industrial[i] * e[i] for i in 1:4) +
    sum(custo_transporte[i][j] * u[i, j] for i in 1:4, j in 1:6)
)

# Definição da função objetivo, dando os valores de multiplicação a cada uma das variáveis:
# Para as de fabrico, o custo industrial da fábrica. Para as de transporte, o custo de transporte do respetivo caminho.
# A variável "e" é usada como "quantidade produzida" (semelhante a x) para o regime extraordinário, daí o preço ser superior.

42.5 x[1] + 46.75000000000001 e[1] + 45 x[2] + 49.50000000000001 e[2] + 47.5 x[3] + 52.25000000000001 e[3] + 50 x[4] + 55.00000000000001 e[4] + 0.1 u[1,1] + 0.6 u[1,2] + 0.25 u[1,3] + 0.8 u[1,4] + 1.2 u[1,5] + 0.55 u[1,6] + 0.5 u[2,1] + 0.8 u[2,2] + 0.05 u[2,3] + 0.2 u[2,4] + 0.5 u[2,5] + 0.75 u[2,6] + 0.4 u[3,1] + 0.5 u[3,2] + 0.7 u[3,3] + 0.45 u[3,4] + u[3,5] + 0.2 u[3,6] + 0.7 u[4,1] + 0.75 u[4,2] + 0.5 u[4,3] + 0.85 u[4,4] + 0.6 u[4,5] + 0.9 u[4,6]

In [47]:
for i in 1:4
    @constraint(mod, x[i] <= capacidade_producao[i])  # Produção limitada à capacidade total de produção
    @constraint(mod, e[i] <= 0.3 * capacidade_producao[i])  # Regime extraordinário limitado a 30% da capacidade normal
end

for i in 1:4
    @constraint(mod, sum(u[i, j] for j in 1:6) == x[i] + e[i]) # A quantidade transportada deve ser igual à produzida
    # Para que as fábricas não fiquem com cerveja não transportada
end

for j in 1:6
    @constraint(mod, sum(u[i, j] for i in 1:4) == procura[j]) # A quantidade transportada deve satisfazer a procura
end

@constraint(mod, u[2, 5] >= 5000)  # Transporta-se pelo menos 5000 de catumbela para namibe
@constraint(mod, u[4, 4] >= 10000)  # Transporta-se pelo menos 10000 de nocebo para huambo
@constraint(mod, x[1] >= 175000)  # Produz-se pelo menos 175000 em Nocal
@constraint(mod, u[1, 3] - u[2, 2] == 0)  # Transporte de Nocal para Lobito deve ser igual ao de Catumbela para Cabinda

# Nenhuma das restrições foi dada nome neste caso, pelo uso de ciclos (dá problemas tentar dar nomes únicos com 'symbol')

-u[2,2] + u[1,3] == 0

In [48]:
print(mod) # Ver o modelo inteiro

In [49]:
optimize!(mod) # Solução ótima: 20 630 551.65

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [5e-02, 6e+01]
  Bound  [0e+00, 0e+00]
  RHS    [5e+03, 2e+05]
Presolving model
10 rows, 31 cols, 56 nonzeros  0s
10 rows, 31 cols, 56 nonzeros  0s
Presolve : Reductions: rows 10(-12); columns 31(-1); elements 56(-13)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     7.4487254882e+06 Pr: 9(630743) 0s
         16     2.0630551650e+07 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 16
Objective value     :  2.0630551650e+07
HiGHS run time      :          0.00


In [50]:
# Resumos das variáveis para análise

println("Custo total: ", objective_value(mod)) # Solução ótima
for i in 1:4
    println("Produção na fábrica $i: ", value(x[i])) # Quantidade produzida em cada fábrica
    println("Produção extraordinária na fábrica $i: ", value(e[i])) # Quantidade produzida em regime extraordinário
end
for j in 1:6
    println("Transporte para o armazém $j: ", [value(u[i, j]) for i in 1:4]) # Quantidades transportadas para cada armazém
end

Custo total: 2.063055165e7
Produção na fábrica 1: 200000.0
Produção extraordinária na fábrica 1: 60000.0
Produção na fábrica 2: 50000.0
Produção extraordinária na fábrica 2: 10743.0
Produção na fábrica 3: 125000.0
Produção extraordinária na fábrica 3: 0.0
Produção na fábrica 4: 10000.0
Produção extraordinária na fábrica 4: 0.0
Transporte para o armazém 1: [245000.0, 0.0, 0.0, 0.0]
Transporte para o armazém 2: [15000.0, -0.0, 15000.0, 0.0]
Transporte para o armazém 3: [0.0, 55743.0, 2644.0, 0.0]
Transporte para o armazém 4: [0.0, 0.0, 48656.0, 10000.0]
Transporte para o armazém 5: [0.0, 5000.0, 20000.0, 0.0]
Transporte para o armazém 6: [0.0, 0.0, 38700.0, 0.0]
